In [ ]:
import numpy as np
import tensorflow as tf
import pathlib


In [ ]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos.tar', origin=dataset_url, extract=True)
data_dir = pathlib.Path(data_dir).with_suffix('')

228813984/228813984 [==============================] - 6s 0us/step


In [ ]:
#data params
batch_size = 32
img_height = 180
img_width = 180

#train set
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

#validation set
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Found 3670 files belonging to 5 classes.
Using 734 files for validation.


In [ ]:
class_names=train_ds.class_names
print(class_names)
num_classes=len(class_names)

['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
data_augmentation = tf.keras.Sequential(
  [
    tf.keras.layers.RandomFlip("horizontal", input_shape=(img_height, img_width,3)),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
  ]
)

In [ ]:
model = tf.keras.Sequential([
  data_augmentation,
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes, name="outputs")
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
epochs = 15
history = model.fit(train_ds, validation_data=val_ds, epochs=epochs)

Epoch 1/15
92/92 [==============================] - 9s 20ms/step - loss: 1.3062 - accuracy: 0.4401 - val_loss: 1.0741 - val_accuracy: 0.5736
Epoch 2/15
92/92 [==============================] - 1s 10ms/step - loss: 1.0382 - accuracy: 0.5943 - val_loss: 0.9837 - val_accuracy: 0.6308
Epoch 3/15
92/92 [==============================] - 1s 11ms/step - loss: 0.9501 - accuracy: 0.6349 - val_loss: 0.8995 - val_accuracy: 0.6431
Epoch 4/15
92/92 [==============================] - 1s 10ms/step - loss: 0.8710 - accuracy: 0.6638 - val_loss: 0.9444 - val_accuracy: 0.6213
Epoch 5/15
92/92 [==============================] - 1s 10ms/step - loss: 0.8105 - accuracy: 0.6918 - val_loss: 0.8679 - val_accuracy: 0.6798
Epoch 6/15
92/92 [==============================] - 1s 10ms/step - loss: 0.7790 - accuracy: 0.7006 - val_loss: 0.8284 - val_accuracy: 0.6717
Epoch 7/15
92/92 [==============================] - 1s 10ms/step - loss: 0.7246 - accuracy: 0.7204 - val_loss: 0.7864 - val_accuracy: 0.6798
Epoch 8/15
92

In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('flower_model.tflite', 'wb') as f:
  f.write(tflite_model)